---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [8]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

import warnings
warnings.filterwarnings('ignore')

In [39]:
def date_sorter():
    # Your code here
    # This was a hard assignment for me so I decided to follow some of the advice in the forums and 
    # break up the regex into six different sections, using the suggestions above.
    # Here is a list of some of the variants you might encounter in this dataset:

    # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    # Feb 2009; Sep 2009; Oct 2010
    # 6/2008; 12/2009
    # 2009; 2010

    #Create six series and perform the above regexes, combining where possible.
    #Erase the extracted date lines from original dataframe (ex)
    df1 = pd.Series(df).str.extractall(r'(?P<month>\d{1,2})[/|-](?P<day>\d{1,2})[/|-](?P<year>\d{2,4})')
    df1.index = df1.index.droplevel(level = 1)
    ex = df[~df.index.isin(df1.index)]
    df1.reset_index(drop=False, inplace=True)

    df2 = pd.Series(df).str.extractall(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.\,\-]* (?P<day>\d{1,2})[[th|st|nd|\-|\-|\,|\.]* (?P<year>\d{2,4})')
    df2.index = df2.index.droplevel(level = 1)
    ex = ex[~ex.index.isin(df2.index)]
    df2.reset_index(drop=False, inplace=True)

    df3 = pd.Series(ex).str.extractall(r'(?P<day>\d{1,2}) (?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[\, |\. | ](?P<year>\d{4})')
    df3.index = df3.index.droplevel(level = 1)
    ex = ex[~ex.index.isin(df3.index)]
    df3.reset_index(drop=False, inplace=True)

    df4 = pd.Series(ex).str.extractall(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.,]* (?P<year>\d{4})')
    df4.index = df4.index.droplevel(level = 1)
    ex = ex[~ex.index.isin(df4.index)]
    df4.reset_index(drop=False, inplace=True)

    df5 = pd.Series(ex).str.extractall(r'(?P<month>\d{1,2})/(?P<year>\d{2,4})')
    df5.index = df5.index.droplevel(level = 1)
    ex = ex[~ex.index.isin(df5.index)]
    df5.reset_index(drop=False, inplace=True)

    df6 = pd.Series(ex).str.extractall(r'(?P<year>\d{4})')
    df6.index = df6.index.droplevel(level = 1)
    ex = ex[~ex.index.isin(df6.index)]
    df6.reset_index(drop=False, inplace=True)

    #Merge these series together
    dates = (df1.merge(df2, on=['index','year','month','day'], how='outer')
                  .merge(df3,on=['index','year','month','day'],how='outer')
                  .merge(df4,on=['index','year','month'],how='outer')
                  .merge(df5,on=['index','year','month'],how='outer')
                  .merge(df6,on=['index','year'],how='outer'))
    
    #Replace month representation with numbers
    dates = dates.replace(to_replace={'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr':'04', 
                                          'May':'05', 'Jun': '06', 'Jul':'07', 'Aug':'08', 
                                          'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'})
    
    #Fill empty(nan) month and day rows with 01 as instructed above
    dates.fillna('01', inplace=True)
    dates.reset_index(drop=True, inplace=True)
    
    #Deal with abberant entry found on visual inspection
    dates = dates.drop(labels=(73), axis=0)
    dates.reset_index(drop=True, inplace=True)

    #Add '19' to year representations from last century that only have two numbers as instructed above
    import re
    i=0
    for year in dates['year']:
        if len(year)==2:
            dates['year'].iloc[i]=re.sub(r'(\d{2})', r'19\1', year)
        i=i+1
    
    #Housekeeping
    dates.index = dates['index']
    dates = dates[['day', 'month','year']]
    dates = pd.to_datetime(dates)
    
    #Sort the values
    dates_sorted = dates.sort_values()
  
    #Converge into final series
    dates_sorted = pd.DataFrame(dates_sorted)
    dates_sorted.reset_index(drop=False, inplace=True)
    dates_sorted = dates_sorted['index']
  
    return dates_sorted

date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     415
17     335
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    480
477    244
478    286
479    431
480    279
481    198
482    463
483    381
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: index, Length: 500, dtype: int64